In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
#Kütüphanelerin eklenmesi
import numpy as np #Bu kütüphane lineer cebir için kullandığımız kütüphane fonksiyonlarını içeriyor
import pandas as pd # verilerimizi işlemek için pandas kütüphanasini kullanıyoruz(örn pd.read_scv)
from sklearn.metrics import *
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

from utils import *
import json
import random
#from spacy.tokenizer import Tokenizer
from spacy.lang.tr import Turkish

from tqdm.notebook import tqdm
from nltk.corpus import stopwords
import re
import string
from sklearn.metrics import classification_report, accuracy_score, make_scorer


from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import *

# DEEP LEARNING IMPORTS
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, Activation, Dropout, Flatten, MaxPooling2D,Bidirectional
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix


from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from sklearn.pipeline import Pipeline


In [2]:
#column = ['tweets','duygu','preprocessing']
#df = pd.read_excel("../dataset/total.xlsx")

column = ['tweets','duygu']
df = pd.read_excel("../dataset/kemik_pos_neg.xlsx")


df.columns=column
#veri setinin gösterilmesi
df=df.drop_duplicates()
df['tweets']=df['tweets'].str.lower()


In [3]:
df.loc[df.duygu=="olumlu","duygu"]=1
df.loc[df.duygu=="olumsuz","duygu"]=0
df[:4]

,tweets,duygu
0,turkcell heryerde çekiyor kesin bilgi,1
1,turkcell olmak ayrıcalıktir çünkü kuzenlerin v...,1
2,allahtan turkcell'liyim amin,1
3,avea kaşar yaşasın turkcell,1


In [4]:
df=df.sample(frac=1).reset_index(drop=True)
df

,tweets,duygu
0,avea yüzünden turkcell in interneti sorunlu di...,1
1,turkcell bana trip atıyor resmen alt tarafı bi...,0
2,turkcell'le ciddi düşünüyorum. ne bilim bana ö...,1
3,sevgi̇li̇m turkcell yeni̇ hat aldiğim i̇çi̇n 1...,1
4,turkcell 4.5 g istemiyorum ben ya 3g bile çalı...,0
...,...,...
9121,turkcell salla kazan 10 gb nasıl bitecek la bu,1
9122,@kaan_terzioglu turkcell e hic bir borcum olma...,0
9123,turkcell yine çekmiyor. aldıkları para boşuna,0
9124,30 tlik fatura yüzünden hattımı aramalara kapa...,0


In [5]:
inputs=df['tweets'].to_numpy()
targets=df['duygu'].to_numpy()

In [6]:
def tf_idf (X_train,y_train,X_test,y_test):
    #Create a tokenizer, configured to only take into account the 20 most common words çok küçük olursa kelimeleri 
    #kaybederiz underfit yaparız
    tokenizer = Tokenizer(num_words=1000) #en yaygın kaç kelimeyi dikkate alacağı. Belirtilecek en iyi kelime sayısı #1000 yapan da var
    tokenizer.fit_on_texts(X_train) #keras tokenizer ile metni dictionary haline getiriyor.
    sequences_X_train = tokenizer.texts_to_sequences(X_train) #kelimelerin dictionarydeki karşılığı 
    #[[2, 1, 3], [2, 1], [4, 1], [5, 6]] şekline getiriliyor. 2-machine 1- learning 3-Knowledge 
    word_index = tokenizer.word_index #dictionarydeki kelimelerin sayısal karşılığı 'unk': 1, 'ürün': 2,
    max_length = 0
    for review_number in range(len(sequences_X_train)): #len(sequences_X_train) ile kaç tane [[2,3,4],[2,6]] var bulunuyor burda 2
        numberofwords=len(sequences_X_train[review_number]) #[2,3,4] içinde kaç tane şey var 3 burda
        if (numberofwords) > (max_length):
            max_length = numberofwords #tüm kelimelere bakıp en uzun olanı buluyor

    vocabulary=[]
    for key in word_index.keys():
        vocabulary.append(key)

    pipe = Pipeline([('count', CountVectorizer(vocabulary=vocabulary)),
                     ('tfid', TfidfTransformer())]).fit(X_train)

    res = dict(zip(vocabulary, pipe['tfid'].idf_))

    X_train = pad_sequences(sequences_X_train, maxlen=max_length) #ikili boyutlu matrise çevirip her cümelnin uzunluğunu eşit yapıyor.
    #En uzun cümle uzunluğuna tamamlanıyor.[[2 1 3] [0 2 1]] alt alta gelecek şekilde en uzun 6 ise 6x6 matris oluyor
    y_train = np.asarray(y_train) #tek boyutlu bir matris oluyor [1 1 0 ... 0 1 0] gibi

    sequences_X_test = tokenizer.texts_to_sequences(X_test) #train için yapılan gibi dictionary alınıyor
    X_test = pad_sequences(sequences_X_test, maxlen=max_length) #en uzun olana göre pad sequence yapılıyor
    y_test = np.asarray(y_test)


    unique_words = len(word_index) #word_index ile unique olan kelimeler alınıyor 0 dan başladığı için bir arttırılıyor
    total_words = unique_words + 1
    
    skipped_words = 0
    embedding_dim = 1 #embedding dim vector size ile aynı 
    embedding_vector=0
    embedding_matrix = np.zeros((total_words, embedding_dim))
    for word, index in tokenizer.word_index.items(): #kelime ve kelimenin dictionarydeki karşılığı alınıyor
        try:
            embedding_vector = res[word] #kelimenin word2vec karşılığı vektör olarak
        except:
            skipped_words = skipped_words+1
            pass
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector 
            
    embedding_layer = Embedding(total_words, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False)

    return embedding_layer,X_train,y_train,X_test,y_test

In [7]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc,pre,rec,f1=[],[],[],[]

pat = 5
early_stopping = EarlyStopping(monitor='loss', patience=pat, verbose=1)

for train, test in kfold.split(inputs, targets):
    
    X_train=inputs[train].tolist()
    y_train=targets[train].tolist()
    X_test=inputs[test].tolist()
    y_test=targets[test].tolist()
    
    embedding_layer,X_train,y_train,X_test,y_test= tf_idf (X_train,y_train,X_test,y_test)
    

    modelLSTM = Sequential()
    modelLSTM.add(embedding_layer)
    modelLSTM.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
    modelLSTM.add(Dense(1, activation='sigmoid'))
    modelLSTM.compile(loss='binary_crossentropy', optimizer='adam', 
                         metrics=['acc',tf.keras.metrics.Precision(),
                                  tf.keras.metrics.Recall()]) #binary cross çünkü sonucun pozitif yada negatif

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    
    modelLSTM.fit(X_train, y_train, epochs=50,callbacks=early_stopping)

    # evaluate
    loss, accuracy, precision, recall = modelLSTM.evaluate(X_test, y_test)
    f1_score = 2 * (precision * recall) / (precision + recall)
    acc.append(accuracy)
    pre.append(precision)
    rec.append(recall)
    f1.append(f1_score)
    
    
    # Increase fold number
    fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/50
257/257 [==============================] - 10s 38ms/step - loss: 0.6953 - acc: 0.5096 - precision: 0.5093 - recall: 0.5893
Epoch 2/50
257/257 [==============================] - 10s 38ms/step - loss: 0.6927 - acc: 0.5156 - precision: 0.5171 - recall: 0.5113
Epoch 3/50
257/257 [==============================] - 10s 40ms/step - loss: 0.6936 - acc: 0.5065 - precision: 0.5071 - recall: 0.5521
Epoch 4/50
257/257 [==============================] - 10s 39ms/step - loss: 0.6926 - acc: 0.5202 - precision: 0.5201 - recall: 0.5523
Epoch 5/50
257/257 [==============================] - 9s 35ms/step - loss: 0.6923 - acc: 0.5182 - precision: 0.5185 - recall: 0.5441
Epoch 6/50
257/257 [==============================] - 9s 35ms/step - loss: 0.6927 - acc: 0.5144 - precision: 0.5159 - recall: 0.5072
Epoch 7/50
257/257 [==============================] - 10s 37ms/step - loss: 0.6921 - acc: 0.5161 - pr

257/257 [==============================] - 15s 56ms/step - loss: 0.6887 - acc: 0.5400 - precision_1: 0.5421 - recall_1: 0.50131s - loss: 0.6893 - acc: 0.5388 - precis - ETA: 0s - loss: 0.6888 - acc: 0.5403 - precision_1: 0.5420 - re
Epoch 24/50
257/257 [==============================] - 16s 63ms/step - loss: 0.6901 - acc: 0.5298 - precision_1: 0.5288 - recall_1: 0.5267
Epoch 25/50
257/257 [==============================] - 18s 69ms/step - loss: 0.6885 - acc: 0.5376 - precision_1: 0.5396 - recall_1: 0.4977
Epoch 26/50
257/257 [==============================] - 19s 73ms/step - loss: 0.6900 - acc: 0.5385 - precision_1: 0.5385 - recall_1: 0.5240
Epoch 27/50
257/257 [==============================] - 20s 76ms/step - loss: 0.6899 - acc: 0.5334 - precision_1: 0.5347 - recall_1: 0.4982
Epoch 28/50
257/257 [==============================] - 18s 71ms/step - loss: 0.6897 - acc: 0.5309 - precision_1: 0.5287 - recall_1: 0.5494
Epoch 29/50
257/257 [==============================] - 18s 71ms/step - l

257/257 [==============================] - 14s 56ms/step - loss: 0.6883 - acc: 0.5455 - precision_3: 0.5471 - recall_3: 0.53793s - loss: 0.6881 - acc: 0.5456 - precision_
Epoch 28/50
257/257 [==============================] - 15s 57ms/step - loss: 0.6890 - acc: 0.5428 - precision_3: 0.5437 - recall_3: 0.54230s - loss: 0.6890 - acc: 0.5418 - precis
Epoch 29/50
257/257 [==============================] - 16s 60ms/step - loss: 0.6896 - acc: 0.5260 - precision_3: 0.5270 - recall_3: 0.5241
Epoch 30/50
257/257 [==============================] - 18s 69ms/step - loss: 0.6900 - acc: 0.5343 - precision_3: 0.5357 - recall_3: 0.5277
Epoch 31/50
257/257 [==============================] - 18s 69ms/step - loss: 0.6898 - acc: 0.5355 - precision_3: 0.5374 - recall_3: 0.5219
Epoch 32/50
257/257 [==============================] - 18s 70ms/step - loss: 0.6903 - acc: 0.5262 - precision_3: 0.5280 - recall_3: 0.5117
Epoch 00032: early stopping
29/29 [==============================] - 0s 8ms/step - loss: 0.692

257/257 [==============================] - 14s 55ms/step - loss: 0.6908 - acc: 0.5264 - precision_5: 0.5270 - recall_5: 0.5175
Epoch 27/50
257/257 [==============================] - 14s 56ms/step - loss: 0.6885 - acc: 0.5387 - precision_5: 0.5378 - recall_5: 0.5528
Epoch 28/50
257/257 [==============================] - 14s 55ms/step - loss: 0.6895 - acc: 0.5320 - precision_5: 0.5310 - recall_5: 0.5497
Epoch 29/50
257/257 [==============================] - 14s 56ms/step - loss: 0.6890 - acc: 0.5363 - precision_5: 0.5375 - recall_5: 0.52341s - loss:
Epoch 30/50
257/257 [==============================] - 15s 57ms/step - loss: 0.6886 - acc: 0.5335 - precision_5: 0.5334 - recall_5: 0.5387
Epoch 00030: early stopping
29/29 [==============================] - 0s 7ms/step - loss: 0.6932 - acc: 0.5235 - precision_5: 0.5610 - recall_5: 0.2022
------------------------------------------------------------------------
Training for fold 7 ...
Epoch 1/50
257/257 [==============================] - 14s 5

257/257 [==============================] - 14s 55ms/step - loss: 0.6903 - acc: 0.5279 - precision_8: 0.5273 - recall_8: 0.5121
Epoch 26/50
257/257 [==============================] - 14s 56ms/step - loss: 0.6900 - acc: 0.5225 - precision_8: 0.5204 - recall_8: 0.5380
Epoch 27/50
257/257 [==============================] - 14s 55ms/step - loss: 0.6895 - acc: 0.5320 - precision_8: 0.5303 - recall_8: 0.5368
Epoch 28/50
257/257 [==============================] - 15s 57ms/step - loss: 0.6903 - acc: 0.5264 - precision_8: 0.5269 - recall_8: 0.49081s - loss: 0.6898 - 
Epoch 29/50
257/257 [==============================] - 14s 55ms/step - loss: 0.6899 - acc: 0.5297 - precision_8: 0.5292 - recall_8: 0.5128
Epoch 30/50
257/257 [==============================] - ETA: 0s - loss: 0.6903 - acc: 0.5304 - precision_8: 0.5300 - recall_8: 0.51 - 14s 56ms/step - loss: 0.6903 - acc: 0.5304 - precision_8: 0.5300 - recall_8: 0.5136
Epoch 31/50
257/257 [==============================] - 14s 56ms/step - loss: 0.6

In [8]:
def Average(lst):
    return sum(lst) / len(lst)

In [9]:
print('test Accuracy: %f' % (Average(acc)))
print('test precision: %f' % (Average(pre)))
print('test recall: %f' % (Average(rec)))
print('test f1_score: %f' % (Average(f1)))

test Accuracy: 0.531338
test precision: 0.556761
test recall: 0.362485
test f1_score: 0.407871
